In [46]:
import json
import os

In [47]:
triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/triples_v2/"
mappings_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/triples_v2/"
filtered_triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples/"
filtered_triples_single_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples_single/"
filtered_triples_type_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples_type/"
filtered_triples_weighted_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples_weighted/"

In [48]:
edges_to_keep = ["http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLink", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://purl.org/dc/terms/subject", "http://www.w3.org/2004/02/skos/core#broader"]
edges_to_delete = ["http://dbkwik.webdatacommons.org/ontology/Image", "http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLinkText"]
files_to_delete = "/File:"

In [49]:
triples = sorted([elem for elem in os.listdir(triples_path) if ".triples" in elem])
mappings = sorted([elem for elem in os.listdir(triples_path) if ".json" in elem])
filtered_triples = sorted([elem for elem in os.listdir(filtered_triples_path) if ".triples" in elem])
filtered_triples_single = sorted([elem for elem in os.listdir(filtered_triples_single_path) if ".triples" in elem])
filtered_triples_type = sorted([elem for elem in os.listdir(filtered_triples_type_path) if ".triples" in elem])
filtered_triples_weighted = sorted([elem for elem in os.listdir(filtered_triples_weighted_path) if ".triples" in elem])
num_of_files = len(triples)

In [41]:
for i in range(num_of_files):
    
    triple = triples[i]
    mapping = mappings[i]
    
    edges_to_keep_num = []
    edges_to_delete_num = []
    
    kept = 0
    deleted = 0
    
    with open(triples_path + triple, "r") as t, open(mappings_path + mapping, "r") as m, open(filtered_triples_path + triple, "w") as f:
        
        data = json.load(m)
        for keep in edges_to_keep:
            edges_to_keep_num.append(str(data.get(keep)))
        for delete in edges_to_delete:
            edges_to_delete_num.append(str(data.get(delete)))
        # print(edges_to_keep_num)
        # print(edges_to_delete_num)
        filtered_data = {key: value for key, value in data.items() if files_to_delete in key}
        for key, value in filtered_data.items():
            edges_to_delete_num.append(str(value))
        # print(len(edges_to_delete_num))
        
        for line in t:
            triple_list = line.replace("\n", "").split("###")
            # print(triple_list)
            if any(edge in triple_list for edge in edges_to_keep_num) and not any(edge in triple_list for edge in edges_to_delete):
                f.write(line)
                kept += 1
            else:
                deleted += 1
        print(f"{triple}: {deleted}/{kept + deleted} triples are deleted")

marvel.triples: 3544412/6733366 triples are deleted
mcu.triples: 678855/1225115 triples are deleted
memoryalpha.triples: 837481/2526928 triples are deleted
memorybeta.triples: 829319/2542909 triples are deleted
starwars.triples: 3457471/8246033 triples are deleted
stexpanded.triples: 245270/567386 triples are deleted
swg.triples: 127665/254354 triples are deleted
swtor.triples: 71635/146148 triples are deleted


In [42]:
for i in range(num_of_files):

    triple = triples[i]
    filtered_triple = filtered_triples[i]
    nodes = set()
    nodes_filtered = set()
    lines = 0
    lines_filtered = 0

    with open(triples_path + triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = line.replace("\n", "").split("###")
            nodes.add(triple_list[0])
            nodes.add(triple_list[1])

    with open(filtered_triples_path + filtered_triple, "r") as t:
        for line in t:
            lines_filtered += 1
            triple_list = line.replace("\n", "").split("###")
            nodes_filtered.add(triple_list[0])
            nodes_filtered.add(triple_list[1])
    
    print(f"{triple}: {len(nodes)} --> {len(nodes_filtered)} nodes ### {lines} --> {lines_filtered} edges")

marvel.triples: 2222423 --> 1152309 nodes ### 6733366 --> 3188954 edges
mcu.triples: 272537 --> 155378 nodes ### 1225115 --> 546260 edges
memoryalpha.triples: 480507 --> 192464 nodes ### 2526928 --> 1689447 edges
memorybeta.triples: 443280 --> 187661 nodes ### 2542909 --> 1713590 edges
starwars.triples: 1158873 --> 440402 nodes ### 8246033 --> 4788562 edges
stexpanded.triples: 125566 --> 45807 nodes ### 567386 --> 322116 edges
swg.triples: 80400 --> 38460 nodes ### 254354 --> 126689 edges
swtor.triples: 39684 --> 18125 nodes ### 146148 --> 74513 edges


In [43]:
for i in range(num_of_files):

    filtered_triple = filtered_triples[i]
    edges = set()
    lines = 0

    with open(filtered_triples_path + filtered_triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = line.replace("\n", "").split("###")
            if str(triple_list[1]) + "###" + str(triple_list[0]) not in edges:
                edges.add(str(triple_list[0]) + "###" + str(triple_list[1]))
            else:
                # print(str(triple_list[0]) + "###" + str(triple_list[1]))
                pass
                
    print(f"{filtered_triple}: {lines} edges with {lines - len(edges)} duplicates (without type) --> {len(edges)} edges")

    with open(filtered_triples_single_path + filtered_triple, "w") as fs:
        for edge in edges:
            fs.write(edge + "\n")

marvel.triples: 3188954 edges with 256580 duplicates (without type) --> 2932374 edges
mcu.triples: 546260 edges with 122238 duplicates (without type) --> 424022 edges
memoryalpha.triples: 1689447 edges with 249634 duplicates (without type) --> 1439813 edges
memorybeta.triples: 1713590 edges with 257906 duplicates (without type) --> 1455684 edges
starwars.triples: 4788562 edges with 970938 duplicates (without type) --> 3817624 edges
stexpanded.triples: 322116 edges with 53814 duplicates (without type) --> 268302 edges
swg.triples: 126689 edges with 22033 duplicates (without type) --> 104656 edges
swtor.triples: 74513 edges with 17395 duplicates (without type) --> 57118 edges


In [44]:
for i in range(num_of_files):

    filtered_triple = filtered_triples[i]
    edges = set()
    lines = 0

    with open(filtered_triples_path + filtered_triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = line.replace("\n", "").split("###")
            if str(triple_list[1]) + "###" + str(triple_list[0]) + '###' + str(triple_list[2]) not in edges:
                edges.add(str(triple_list[0]) + "###" + str(triple_list[1]) + '###' + str(triple_list[2]))
            else:
                # print(str(triple_list[0]) + "###" + str(triple_list[1]) + '###' + str(triple_list[2])
                pass

    print(f"{filtered_triple}: {lines} edges with {lines - len(edges)} duplicates (with type) --> {len(edges)} edges")

    with open(filtered_triples_type_path + filtered_triple, "w") as fs:
        for edge in edges:
            fs.write(edge + "\n")

marvel.triples: 3188954 edges with 108113 duplicates (with type) --> 3080841 edges
mcu.triples: 546260 edges with 62368 duplicates (with type) --> 483892 edges
memoryalpha.triples: 1689447 edges with 180331 duplicates (with type) --> 1509116 edges
memorybeta.triples: 1713590 edges with 126354 duplicates (with type) --> 1587236 edges
starwars.triples: 4788562 edges with 678447 duplicates (with type) --> 4110115 edges
stexpanded.triples: 322116 edges with 26048 duplicates (with type) --> 296068 edges
swg.triples: 126689 edges with 7559 duplicates (with type) --> 119130 edges
swtor.triples: 74513 edges with 3156 duplicates (with type) --> 71357 edges


In [45]:
for i in range(num_of_files):

    triple = triples[i]
    filtered_triple_type = filtered_triples_type[i]
    nodes = set()
    nodes_filtered_type = set()
    lines = 0
    lines_filtered_type = 0

    with open(triples_path + triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = line.replace("\n", "").split("###")
            nodes.add(triple_list[0])
            nodes.add(triple_list[1])

    with open(filtered_triples_type_path + filtered_triple_type, "r") as t:
        for line in t:
            lines_filtered_type += 1
            triple_list = line.replace("\n", "").split("###")
            nodes_filtered_type.add(triple_list[0])
            nodes_filtered_type.add(triple_list[1])

    print(f"{triple}: {len(nodes)} --> {len(nodes_filtered_type)} nodes ### {lines} --> {lines_filtered_type} edges")

marvel.triples: 2222423 --> 1152309 nodes ### 6733366 --> 3080841 edges
mcu.triples: 272537 --> 155378 nodes ### 1225115 --> 483892 edges
memoryalpha.triples: 480507 --> 192464 nodes ### 2526928 --> 1509116 edges
memorybeta.triples: 443280 --> 187661 nodes ### 2542909 --> 1587236 edges
starwars.triples: 1158873 --> 440402 nodes ### 8246033 --> 4110115 edges
stexpanded.triples: 125566 --> 45807 nodes ### 567386 --> 296068 edges
swg.triples: 80400 --> 38460 nodes ### 254354 --> 119130 edges
swtor.triples: 39684 --> 18125 nodes ### 146148 --> 71357 edges


In [54]:
for i in range(num_of_files):
    
    print("Processing file", i + 1, "of", num_of_files)

    filtered_triple = filtered_triples[i]
    edges = dict()
    lines = 0

    with open(filtered_triples_path + filtered_triple, "r") as t:
        for line in t:
            lines += 1
            triple_list = sorted(line.replace("\n", "").split("###")[:2])
            key = str(triple_list[0]) + "###" + str(triple_list[1])
            edges[key] = edges.get(key, 0) + 1

    with open(filtered_triples_weighted_path + filtered_triple, "w") as fw:
        for k, v in edges.items():
            fw.write(str(k) + "###" + str(v) + "\n")

Processing file 1 of 8
Processing file 2 of 8
Processing file 3 of 8
Processing file 4 of 8
Processing file 5 of 8
Processing file 6 of 8
Processing file 7 of 8
Processing file 8 of 8
